# Evaluating `pyDeid`

We show how to evaluate `pyDeid` on a custom dataset of Canadian admission notes, as described in the paper

> pyDeid: An Improved, Fast, Flexible, and Generalizable Rules-based Approach for De-identification of Free-text Medical Records

As well as on the popular `n2c2` benchmark dataset of American discharge notes, using the [`ETUDE engine`](https://github.com/MUSC-TBIC/etude-engine).

## Testing CSV output against a gold standard dataset with `CSVEvaluator`

Given a test dataset of clinical notes formatted in a `csv` such as that found in `tests/test.csv`, and a "gold standard" dataset with each note split by token, and annotated with the appropriate PII type, we can evaluate the performance of `pyDeid` using the `CSVEvaluator` class.

First we run `pyDeid` on `tests/test.csv`, outputting in `csv` format.

In [4]:
from pyDeid import pyDeid

pyDeid(
    original_file = "./tests/test.csv",
    note_varname = "note_text",
    encounter_id_varname = "genc_id",
    note_id_varname = "note_id"
)

Processing encounter 3, note Record 3: : 3it [00:00,  4.10it/s]

Diagnostics:
                - chars/s = 178.9151600603055
                - s/note = 0.24406353632609049


From here we use the annotated ground truth dataset.

We show how to create a ground truth CSV given raw notes in the form of `./tests/test.csv`.

1. Begin by tokenizing the raw notes using `tokenize_csv()`.
2. Manually annotate the notes for PHI.
3. Using `melt_annotations()`, combine multi-token PHI into a single entry.

In particular dates and locations are usually multi-token, so we handle them specifically.

In [ ]:
from pyDeid.phi_types import tokenize_csv, melt_annotations

tokenize_csv(
    input_file = "./tests/test.csv",
    output_file = "./tests/ground_truth.csv",
    encounter_id_varname = "genc_id",
    note_id_varname = "note_id",
    note_text_varname = "note_text"
)

Then we annotate the file `./tests/ground_truth.csv`.

Once that is complete, we combine multi-token PHI.

In [ ]:
melt_annotations(
    input_file = "./tests/ground_truth.csv",
    output_file = "./tests/ground_truth_processed.csv",
    merge_annotations = ["d", "l"]
)

Now we have a ground truth dataset that we can use to compare against the output of `pyDeid`.

To do this we use the `CSVEvaluator` class.

In [ ]:
from pyDeid.phi_types.CSVEvaluator import CSVEvaluator

evaluator = CSVEvaluator()

evaluator.add_ground_truth_file("./tests/ground_truth.csv", note_id_varname = "note_id")\
    .add_result_file("./tests/test__PHI.csv")\
    .evaluate()

## Using the ETUDE Engine

The ETUDE engine is a well established, standard tool for analyzing de-identification performance against various benchmark dataset formats, such as the `n2c2` format.

In this section, we use the ETUDE engine to evaluate the performance of `pyDeid` on the `n2c2` dataset. We begin by cloning the repository.

In [ ]:
!git clone git@github.com:MUSC-TBIC/etude-engine.git

In order to run the evaluation on `n2c2`, `pyDeid` does require the ability to read and write from `xml`. This is available through the `pyDeid_n2c2()` function.

Ensure that the `n2c2` dataset is saved to some directory such as `./tests/n2c2`.

In [7]:
from pyDeid.n2c2 import pyDeid_n2c2

pyDeid_n2c2(
    input_dir = "./tests/n2c2",
    output_dir = "./tests/pydeid_n2c2_output",
)

In [ ]:
from pyDeid.n2c2 import pyDeid_n2c2

pyDeid_n2c2(
    input_dir = "./tests/n2c2",
    output_dir = "./tests/pydeid_n2c2_output",
)

Note that this is essentially ready to run the evaluation on, however there is a significant difference between how `pyDeid` recognizes names and how names are annotated in the `n2c2` ground truth. In `pyDeid`, first and last names are considered separately, and so we must separate these annotations in the `n2c2` ground truth using `split_multi_word_tags()`.

In [1]:
from pyDeid import split_multi_word_tags

split_multi_word_tags(
    input_dir = "./tests/n2c2",
    output_dir = "./tests/n2c2_ground_truth_preprocessed"
)

Now we are ready to run the `ETUDE engine`.

In order to compare a given reference file with a given test file, the `ETUDE engine` uses a config for each of the reference file, and the tool's output. We provide both of the relevant configs under the `./tests/ETUDE_configs` directory.

With this, we run the following command from the directory in which we cloned the `ETUDE engine` repository.

In [6]:
!python H:/repos/GitHub/etude-engine/etude.py \
    --reference-input "R:/GEMINI/Research Projects/pyDeid/02_CohortData/n2c2/n2c2_test_data_raw_preprocessed" \
    --reference-config H:/repos/GitHub/etude-engine/config/n2c2_pydeid.conf \
    --test-input "R:/GEMINI/Research Projects/pyDeid/02_CohortData/n2c2/n2c2_test_data_pydeid_output" \
    --test-config H:/repos/GitHub/etude-engine/config/pydeid_n2c2.conf \
    --by-type --score-key "Parent"


exact	TP	FP	TN	FN
micro-average	6170.0	2138.0	0.0	5977.0
Address	0.0	480.0	0.0	1102.0
Contact Information	37.0	63.0	0.0	230.0
Identifiers	82.0	32.0	0.0	571.0
Locations	0.0	120.0	0.0	20.0
Names	3893.0	982.0	0.0	725.0
Time	2158.0	461.0	0.0	3329.0
macro-average by type	6170.0	2138.0	0.0	5977.0



100%|##########| 514/514 [22:34<00:00,  2.64s/it]
